#Introduction

Brian wanted FPKMs for some of the Gingeras datasets that fail MAD.

In [ ]:
import pandas
import os
import sys

In [ ]:
ROOT=os.path.expanduser("~diane/proj/encode3-curation")
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [ ]:
from curation_common import *
import save_rnaseq_madqc

In [ ]:
query_url = 'search/?type=experiment&assay_term_name=RNA-seq'
cache_name = os.path.join(ROOT, 'rnaseq-experiments.shelf')
cache = save_rnaseq_madqc.caching_encoded_experiment_loader(query_url, cache_name)

In [ ]:
experiment_qc = pandas.read_csv(os.path.join(ROOT, 'compare-mad', 'experiment-mad-qc.csv'))

In [ ]:
gingeras = experiment_qc[experiment_qc['lab'] == 'Thomas Gingeras, CSHL']

In [ ]:
gingeras_fail_mad = gingeras[(gingeras['MAD'] > 0.5)]

In [ ]:
gingeras_fail_mad.to_csv(os.path.expanduser('~/public_html/compare-mad/gingeras-fail-mad.csv'))

In [ ]:
def load_spike_rsems(cache, keys, limit=None):
    keys = list(keys)
    total = len(keys)
    chunk = max(total // 10, 1)
    tzero = time.monotonic()
    tprev = tzero
    
    for i, experiment_id in enumerate(keys):
        experiment = cache[experiment_id]
        fpkms = []
        for file in save_rnaseq_madqc.find_rsem(experiment['files']):
            url = 'https://www.encodeproject.org' + file.href
            fpkm = pandas.read_csv(url, usecols=[0,6], sep='\t', index_col=0)
            fpkm.columns = [file.library_id]
            fpkms.append(fpkm)

        if fpkms:
            yield (experiment_id, pandas.concat(fpkms, axis=1))
            
        if (i + 1) % chunk == 0:
            tnow = time.monotonic()
            print("{} of {} in {:.2f} sec".format(
                i, total, tnow-tprev))
            tprev = tnow

        if limit and i > limit:
            return

In [ ]:
for experiment_id, fpkms in load_spike_rsems(
      cache,
      keys=list(gingeras_fail_mad['experiment'])
    ):
    filename = os.path.join(os.path.expanduser('~/public_html/compare-mad/'), experiment_id + '_library_fpkms.csv')
    fpkms.to_csv(filename)